In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn import cluster, preprocessing
%matplotlib inline

# Data

Load data from `nc_complete-2014.csv`.

In [2]:
df = pd.read_csv('nc_complete-2014.csv', index_col=0)
df.head()

,county_name,precinct_id,sen_red,sen_blue,con_districts,con_contested,con_red_votes,con_blue_votes,sldu_districts,sldu_contested,...,hispanic_pop,other_pop,median_age,median_income,education_pop,school_pop,diploma_pop,college_pop,graduate_pop,area_km2
shape_id,,,,,,,,,,,,,,,,,,,,,
2368,ALAMANCE,01,1071,295,2,Yes,1109,313.0,24,No,...,85,108,42,44870,3679,727,1518,1278,156,129
2366,ALAMANCE,02,1246,313,6,Yes,1285,314.0,24,No,...,203,102,41,49485,3525,601,1318,1361,245,77
2361,ALAMANCE,035,920,434,6,Yes,955,431.0,24,No,...,624,204,27,41560,3624,684,1173,1591,176,27
2374,ALAMANCE,03C,943,455,6,Yes,979,447.0,24,No,...,127,44,46,67658,2237,133,557,1167,381,5
2723,ALAMANCE,03N,862,586,6,Yes,881,582.0,24,No,...,85,20,37,67101,2792,92,691,1508,501,5


### Derived Values

Combine contested votes into single percentages, calculate population density, and use population rates instead of raw values.

In [3]:
df = df.assign(density = df.population / df.area_km2.clip(lower=1))

df = df.assign(sen_pctblue = df.sen_blue / (df.sen_blue + df.sen_red))
df = df.assign(sen_turnout = (df.sen_blue + df.sen_red) / df.population)
df = df.assign(con_pctblue = df.con_blue_votes / (df.con_blue_votes + df.con_red_votes))
df = df.assign(con_turnout = (df.con_blue_votes + df.con_red_votes) / df.population)
df = df.assign(sldu_pctblue = df.sldu_blue_votes / (df.sldu_blue_votes + df.sldu_red_votes))
df = df.assign(sldu_turnout = (df.sldu_blue_votes + df.sldu_red_votes) / df.population)
df = df.assign(sldl_pctblue = df.sldl_blue_votes / (df.sldl_blue_votes + df.sldl_red_votes))
df = df.assign(sldl_turnout = (df.sldl_blue_votes + df.sldl_red_votes) / df.population)

df = df.assign(pct_white = df.white_pop / df.population)
df = df.assign(pct_black = df.black_pop / df.population)
df = df.assign(pct_asian = df.asian_pop / df.population)
df = df.assign(pct_hispanic = df.hispanic_pop / df.population)
df = df.assign(pct_other = df.other_pop / df.population)

df = df.assign(pct_school = df.school_pop / df.education_pop)
df = df.assign(pct_diploma = df.diploma_pop / df.education_pop)
df = df.assign(pct_college = df.college_pop / df.education_pop)
df = df.assign(pct_graduate = df.graduate_pop / df.education_pop)

df = df.assign(pct_income = df.median_income / df.median_income.max())
df = df.assign(pct_age = df.median_age / df.median_age.max())

# Learn

In [4]:
from sklearn import datasets, linear_model
import numpy as np

In [5]:
def calc_regr(df, query1, indep_fields, field, test_fields, query2, filename):
    ''' 
    '''
    full_all = df.query(query1).filter(items=indep_fields + (field,)).dropna()
    full_input = full_all.filter(items=indep_fields)
    full_output = full_all.filter(items=(field,))

    assert full_input.shape == full_input.dropna().shape
    assert full_output.shape == full_output.dropna().shape
    
    limited_input = full_all.filter(items=test_fields)
    
    training_size = len(full_input) * 4 // 5
    print('Training set length:', training_size)
    limited_train, limited_test = limited_input[:training_size], limited_input[training_size:]
    output_train, output_test = full_output[:training_size], full_output[training_size:]
    
    regr = linear_model.LinearRegression()
    regr.fit(limited_train, output_train)

    print('Linear regression score: {:.0f}%'.format(100 * regr.score(limited_test, output_test)))
    
    coefficients = sorted([(abs(c), test_fields[i]) for (i, c) in enumerate(regr.coef_[0,:])], reverse=True)
    print('Coefficients:', ', '.join(['{1} ({0:.02f})'.format(c, f) for (c, f) in coefficients if c > .01]))
    
    real_input = df.query(query2).filter(items=test_fields).dropna()
    real_output = real_input.assign(**{field: regr.predict(real_input)}).filter(items=(field, ))
    real_output.to_csv(filename)
    print('Wrote', real_output.shape, filename, 'from', real_input.shape, 'input')

## U.S. House Votes & Turnout

In [6]:
core_fields = ('pct_black', 'pct_income', 'pct_age', 'pct_diploma')
con1_test_fields = core_fields + ('sen_pctblue', )

calc_regr(
    df,
    'con_contested == "Yes"',
    core_fields + ('sen_pctblue', 'sen_turnout'),
    'con_pctblue',
    con1_test_fields,
    'con_contested != "Yes"',
    'con1_real_output.csv')

con2_test_fields = core_fields + ('sen_turnout', )

calc_regr(
    df,
    'con_contested == "Yes"',
    core_fields + ('sen_pctblue', 'sen_turnout'),
    'con_turnout',
    con2_test_fields,
    'con_contested != "Yes"',
    'con2_real_output.csv')

Training set length: 2051
Linear regression score: 96%
Coefficients: sen_pctblue (0.99), pct_diploma (0.04), pct_black (0.04), pct_age (0.02), pct_income (0.01)
Wrote (161, 1) con1_real_output.csv from (161, 5) input
Training set length: 2051
Linear regression score: 98%
Coefficients: sen_turnout (1.05), pct_black (0.12), pct_age (0.08), pct_income (0.02), pct_diploma (0.02)
Wrote (161, 1) con2_real_output.csv from (161, 5) input


/Users/migurski/Sites/Election-Results/.venv/lib/python3.6/site-packages/scipy/linalg/basic.py:1018: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  warnings.warn(mesg, RuntimeWarning)


## SLDU Blue Votes

Pick out all complete list of independent variables plus the SLDU dependent variables: `sldu_pctblue` (SLDU Democratic vote percentage) and `sldu_turnout` (number of voters out of total population). After some trial-and-error with the full list of fields, I saw the best fit results with `pct_black` (Race), `pct_income` (Income), `pct_age` (Age), and `pct_diploma` (Education).

In [7]:
core_fields = ('pct_black', 'pct_income', 'pct_age', 'pct_diploma')
sldu1A_test_fields = core_fields + ('sen_pctblue', 'con_pctblue')

calc_regr(
    df,
    'sldu_contested == "Yes" and con_contested == "Yes"',
    core_fields + ('sen_pctblue', 'con_pctblue', 'sen_turnout', 'con_turnout'),
    'sldu_pctblue',
    sldu1A_test_fields,
    'sldu_contested != "Yes" and con_contested == "Yes"',
    'sldu1A_real_output.csv')

sldu1B_test_fields = core_fields + ('sen_pctblue', )

calc_regr(
    df,
    'sldu_contested == "Yes"',
    core_fields + ('sen_pctblue', 'sen_turnout'),
    'sldu_pctblue',
    sldu1B_test_fields,
    'sldu_contested != "Yes"',
    'sldu1B_real_output.csv')

Training set length: 1179
Linear regression score: 95%
Coefficients: sen_pctblue (1.02), pct_income (0.11), con_pctblue (0.07), pct_diploma (0.06), pct_black (0.01)
Wrote (1090, 1) sldu1A_real_output.csv from (1090, 6) input
Training set length: 1209
Linear regression score: 94%
Coefficients: sen_pctblue (0.96), pct_income (0.14), pct_diploma (0.07), pct_black (0.02)
Wrote (1213, 1) sldu1B_real_output.csv from (1213, 5) input


## SLDU Voter Turnout

In [8]:
core_fields = ('pct_black', 'pct_income', 'pct_age', 'pct_diploma')
sldu2A_test_fields = core_fields + ('sen_turnout', 'con_turnout')

calc_regr(
    df,
    'sldu_contested == "Yes" and con_contested == "Yes"',
    core_fields + ('sen_pctblue', 'con_pctblue', 'sen_turnout', 'con_turnout'),
    'sldu_turnout',
    sldu2A_test_fields,
    'sldu_contested != "Yes" and con_contested == "Yes"',
    'sldu2A_real_output.csv')

sldu2B_test_fields = core_fields + ('sen_turnout', )

calc_regr(
    df,
    'sldu_contested == "Yes"',
    core_fields + ('sen_pctblue', 'sen_turnout'),
    'sldu_turnout',
    sldu2B_test_fields,
    'sldu_contested != "Yes"',
    'sldu2B_real_output.csv')

Training set length: 1179
Linear regression score: 100%
Coefficients: sen_turnout (1.02), pct_income (0.04), pct_black (0.02), pct_diploma (0.02), con_turnout (0.01)
Wrote (1090, 1) sldu2A_real_output.csv from (1090, 6) input
Training set length: 1209
Linear regression score: 100%
Coefficients: sen_turnout (1.03), pct_income (0.04), pct_black (0.02), pct_diploma (0.02)
Wrote (1213, 1) sldu2B_real_output.csv from (1213, 5) input


## SLDL Blue Votes

Pick out all complete list of independent variables plus the SLDL dependent variables: `sldl_pctblue` (SLDL Democratic vote percentage) and `sldl_turnout` (number of voters out of total population).

In [9]:
core_fields = ('pct_black', 'pct_income', 'pct_age', 'pct_diploma')
sldl1A_test_fields = core_fields + ('sen_pctblue', 'con_pctblue')

calc_regr(
    df,
    'sldl_contested == "Yes" and con_contested == "Yes"',
    core_fields + ('sen_pctblue', 'con_pctblue', 'sen_turnout', 'con_turnout'),
    'sldl_pctblue',
    sldl1A_test_fields,
    'sldl_contested != "Yes" and con_contested == "Yes"',
    'sldl1A_real_output.csv')

sldl1B_test_fields = core_fields + ('sen_pctblue', )

calc_regr(
    df,
    'sldl_contested == "Yes"',
    core_fields + ('sen_pctblue', 'sen_turnout'),
    'sldl_pctblue',
    sldl1B_test_fields,
    'sldl_contested != "Yes"',
    'sldl1B_real_output.csv')

Training set length: 992
Linear regression score: 92%
Coefficients: sen_pctblue (1.05), con_pctblue (0.11), pct_income (0.10), pct_black (0.05), pct_diploma (0.04), pct_age (0.01)
Wrote (1324, 1) sldl1A_real_output.csv from (1324, 6) input
Training set length: 1030
Linear regression score: 91%
Coefficients: sen_pctblue (0.95), pct_income (0.09), pct_black (0.05), pct_diploma (0.04), pct_age (0.02)
Wrote (1437, 1) sldl1B_real_output.csv from (1437, 5) input


## SLDL Voter Turnout

In [10]:
core_fields = ('pct_black', 'pct_income', 'pct_age', 'pct_diploma')
sldl2A_test_fields = core_fields + ('sen_turnout', 'con_turnout')

calc_regr(
    df,
    'sldl_contested == "Yes" and con_contested == "Yes"',
    core_fields + ('sen_pctblue', 'con_pctblue', 'sen_turnout', 'con_turnout'),
    'sldl_turnout',
    sldl2A_test_fields,
    'sldl_contested != "Yes" and con_contested == "Yes"',
    'sldl2A_real_output.csv')

sldl2B_test_fields = core_fields + ('sen_turnout', )

calc_regr(
    df,
    'sldl_contested == "Yes"',
    core_fields + ('sen_pctblue', 'sen_turnout'),
    'sldl_turnout',
    sldl2B_test_fields,
    'sldl_contested != "Yes"',
    'sldl2B_real_output.csv')

Training set length: 992
Linear regression score: 100%
Coefficients: sen_turnout (0.94), con_turnout (0.07), pct_diploma (0.05), pct_income (0.03), pct_black (0.03), pct_age (0.02)
Wrote (1324, 1) sldl2A_real_output.csv from (1324, 6) input
Training set length: 1030
Linear regression score: 100%
Coefficients: sen_turnout (1.01), pct_diploma (0.05), pct_black (0.03), pct_income (0.02), pct_age (0.01)
Wrote (1437, 1) sldl2B_real_output.csv from (1437, 5) input


# Merge

Use `csvjoin` to read each of the CSV files created above into a single output file by `shape_id`, impute new SLDU and SLDL votes if necessary, and output a new `nc_imputed-2014.csv` file.

In [11]:
import subprocess, io, csv
bytes = subprocess.check_output((
    'csvjoin', '--left', '-c', 'shape_id', 'nc_complete-2014.csv',
    'con1_real_output.csv', 'con2_real_output.csv',
    'sldu1A_real_output.csv', 'sldu1B_real_output.csv', 'sldu2A_real_output.csv', 'sldu2B_real_output.csv',
    'sldl1A_real_output.csv', 'sldl1B_real_output.csv', 'sldl2A_real_output.csv', 'sldl2B_real_output.csv'
))

with open('nc_imputed-2014.csv', 'w') as file:
    input = csv.DictReader(io.StringIO(bytes.decode('utf8')))
    output = csv.DictWriter(file, input.fieldnames[:-10] + ['con_imputed', 'sldu_imputed', 'sldl_imputed'])
    output.writeheader()
    
    for row in input:
        population = int(row['population'])

        if row['con_contested'] == 'Yes':
            row['con_imputed'] = 'No'
        else:
            con_pctblue = float(row['con_pctblue'])
            con_turnout = float(row['con_turnout'])
            row['con_blue_votes'] = int(con_pctblue * con_turnout * population)
            row['con_red_votes'] = int((1 - con_pctblue) * con_turnout * population)
            row['con_imputed'] = 'Yes'

        if row['sldu_contested'] == 'Yes':
            row['sldu_imputed'] = 'No'
        else:
            sldu_pctblue = float(row['sldu_pctblue'] or row['sldu_pctblue2'])
            sldu_turnout = float(row['sldu_turnout'] or row['sldu_turnout2'])
            row['sldu_blue_votes'] = int(sldu_pctblue * sldu_turnout * population)
            row['sldu_red_votes'] = int((1 - sldu_pctblue) * sldu_turnout * population)
            row['sldu_imputed'] = 'Yes'

        if row['sldl_contested'] == 'Yes':
            row['sldl_imputed'] = 'No'
        else:
            sldl_pctblue = float(row['sldl_pctblue'] or row['sldl_pctblue2'])
            sldl_turnout = float(row['sldl_turnout'] or row['sldl_turnout2'])
            row['sldl_blue_votes'] = int(sldl_pctblue * sldl_turnout * population)
            row['sldl_red_votes'] = int((1 - sldl_pctblue) * sldl_turnout * population)
            row['sldl_imputed'] = 'Yes'
        
        output.writerow({k: row[k] for k in output.fieldnames})

'OK'

'OK'